In [273]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Dropout, MaxPooling1D, Conv1D, LSTM, SimpleRNN
from tensorflow.keras.callbacks import EarlyStopping

In [274]:
# acquire MVP vote shares and box score statistics
mvpData = pd.read_csv("../NBA Stats/Player Award Shares.csv")
boxScore = pd.read_csv("../NBA Stats/Player Per Game.csv")

In [275]:
mvpData = mvpData[mvpData['award'] == 'nba mvp']
# 1980 season marks the beginning of 3-pt era and tracking of 3-pt shooting statistics
mvpData = mvpData[mvpData['season'] >= 1980]
mvpData = mvpData.merge(boxScore, on='seas_id', how='inner')

In [276]:
# create X and Y training data
Y = mvpData['share'].apply(lambda x: x*1000).to_numpy()
X = np.c_[mvpData['age_x'], mvpData['experience'], mvpData['mp_per_game'],
       mvpData['fg_per_game'], mvpData['fga_per_game'], mvpData['x3p_per_game'],
       mvpData['x3pa_per_game'], mvpData['x2p_per_game'], mvpData['x2pa_per_game'], 
       mvpData['ft_per_game'], mvpData['fta_per_game'],  mvpData['orb_per_game'],
       mvpData['drb_per_game'], mvpData['ast_per_game'], mvpData['stl_per_game'],
       mvpData['blk_per_game'], mvpData['tov_per_game'], mvpData['pf_per_game']]

In [277]:
# train NN to predict share of MVP votes
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input

np.random.seed(21)
tf.random.set_seed(21)

early_stopping_callback = EarlyStopping(monitor='loss', patience=3)

model = Sequential()
model.add(Input(shape=(18,)))
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(1, activation="relu"))
model.compile(optimizer="adam", loss="mean_absolute_error")
model.fit(X, Y, epochs=100, batch_size=32, verbose=2, callbacks=[early_stopping_callback])

Epoch 1/100
23/23 - 1s - 38ms/step - loss: 153.3691
Epoch 2/100
23/23 - 0s - 3ms/step - loss: 150.8639
Epoch 3/100
23/23 - 0s - 2ms/step - loss: 149.3803
Epoch 4/100
23/23 - 0s - 3ms/step - loss: 148.2355
Epoch 5/100
23/23 - 0s - 2ms/step - loss: 147.2029
Epoch 6/100
23/23 - 0s - 3ms/step - loss: 145.9141
Epoch 7/100
23/23 - 0s - 3ms/step - loss: 144.8446
Epoch 8/100
23/23 - 0s - 3ms/step - loss: 143.8331
Epoch 9/100
23/23 - 0s - 3ms/step - loss: 142.8185
Epoch 10/100
23/23 - 0s - 2ms/step - loss: 141.4739
Epoch 11/100
23/23 - 0s - 3ms/step - loss: 140.0250
Epoch 12/100
23/23 - 0s - 3ms/step - loss: 139.3293
Epoch 13/100
23/23 - 0s - 2ms/step - loss: 137.9524
Epoch 14/100
23/23 - 0s - 3ms/step - loss: 136.9784
Epoch 15/100
23/23 - 0s - 3ms/step - loss: 135.8507
Epoch 16/100
23/23 - 0s - 3ms/step - loss: 135.1249
Epoch 17/100
23/23 - 0s - 3ms/step - loss: 134.2863
Epoch 18/100
23/23 - 0s - 3ms/step - loss: 133.3067
Epoch 19/100
23/23 - 0s - 3ms/step - loss: 132.2908
Epoch 20/100
23/23 -

In [278]:
#go through top 10 of each season and select highest predicted mvp share for each year
MVPs = []
for year in range(1980, 2025):
        players = mvpData[mvpData['season_x'] == year]
        maxVal = 0
        mvp = players['player_x'].iloc[0]
        for index, row in players.iterrows():
            temp = model.predict(np.c_[row['age_x'], row['experience'], row['mp_per_game'],
                row['fg_per_game'], row['fga_per_game'], row['x3p_per_game'],
                row['x3pa_per_game'], row['x2p_per_game'], row['x2pa_per_game'], 
                row['ft_per_game'], row['fta_per_game'],  row['orb_per_game'],
                row['drb_per_game'], row['ast_per_game'], row['stl_per_game'],
                row['blk_per_game'], row['tov_per_game'], row['pf_per_game']], verbose=0)
            if temp > maxVal:
                maxVal = temp
                mvp = row['player_x']
        tempStr = str(year) + ": " + mvp
        MVPs.append(tempStr)


In [279]:
# print which player had the highest predicted MVP share for each year
print("Predicted MVPs")
for mvp in MVPs:
    print(mvp)


Predicted MVPs
1980: Kareem Abdul-Jabbar
1981: Moses Malone
1982: Moses Malone
1983: Moses Malone
1984: Magic Johnson
1985: Larry Bird
1986: Magic Johnson
1987: Michael Jordan
1988: Michael Jordan
1989: Michael Jordan
1990: Hakeem Olajuwon
1991: David Robinson
1992: David Robinson
1993: Hakeem Olajuwon
1994: David Robinson
1995: Shaquille O'Neal
1996: David Robinson
1997: Shaquille O'Neal
1998: Shaquille O'Neal
1999: Shaquille O'Neal
2000: Shaquille O'Neal
2001: Shaquille O'Neal
2002: Shaquille O'Neal
2003: Tracy McGrady
2004: Kevin Garnett
2005: Kevin Garnett
2006: LeBron James
2007: Dwyane Wade
2008: LeBron James
2009: LeBron James
2010: LeBron James
2011: Dwight Howard
2012: LeBron James
2013: LeBron James
2014: Kevin Durant
2015: Russell Westbrook
2016: Russell Westbrook
2017: Russell Westbrook
2018: James Harden
2019: Giannis Antetokounmpo
2020: Giannis Antetokounmpo
2021: Giannis Antetokounmpo
2022: Giannis Antetokounmpo
2023: Giannis Antetokounmpo
2024: Luka Dončić


The model chose a different MVP in the following years:  
2024 (Model-chosen: Luka Dončić, Voter-chosen: Nikola Jokic)  
2023 (Model-chosen: Giannis Antetokounmpo, Voter-chosen: Joel Embiid)  
2021, 2022 (Model-chosen: Giannis Antetokounmpo, Voter-chosen: Nikola Jokic)  
2015, 2016 (Model-chosen: Russell Westbrook, Voter-chosen: Stephen Curry)  
2011 (Model-chosen: Dwight Howard, Voter-chosen: Derrick Rose)  
2008 (Model-chosen: LeBron James, Voter-chosen: Kobe Bryant)  
2007 (Model-chosen: Dwyane Wade, Voter-chosen: Dirk Nowitzki)  
2006 (Model-chosen: LeBron James, Voter-chosen: Steve Nash)  
2005 (Model-chosen: Kevin Garnett, Voter-chosen: Steve Nash)  
2003 (Model-chosen: Tracy McGrady, Voter-chosen: Tim Duncan)  
2002 (Model-chosen: Shaquille O'Neal, Voter-chosen: Tim Duncan)  
2001 (Model-chosen: Shaquille O'Neal, Voter-chosen: Allen Iverson)  
1997, 1999 (Model-chosen: Shaquille O'Neal, Voter-chosen: Karl Malone)  
1998 (Model-chosen: Shaquille O'Neal, Voter-chosen: Michael Jordan)  
1996 (Model-chosen: David Robinson, Voter-chosen: Michael Jordan)  
1995 (Model-chosen: Shaquille O'Neal, Voter-chosen: David Robinson)  
1994 (Model-chosen: David Robinson, Voter-chosen: Hakeem Olajuwon)  
1993 (Model-chosen: Hakeem Olajuwon, Voter-chosen: Charles Barkley)  
1991, 1992 (Model-chosen: David Robinson, Voter-chosen: Michael Jordan)  
1990 (Model-chosen: Hakeem Olajuwon, Voter-chosen: Magic Johnson)  
1987, 1989 (Model-chosen: Michael Jordan, Voter-chosen: Magic Johnson)  
1984, 1986 (Model-chosen: Magic Johnson, Voter-chosen: Larry Bird)  
1981 (Model-chosen: Moses Malone, Voter-chosen: Julius Erving)  